# **Download high-resolution Daytime satellite images using Planet API**



1) Planet Scope (PSScene): PlanetScope is one of the satellite constellation operated by Planet.

- **Revisit time**: It has almost daily coverage worldwide (1 day).
- **Spatial resolution**: 3.7-4.1 m pixel size (resampled to 3 m)
- **Data availability**: Data availability since 2016
- **Bands**:
   - Four-band Imager (PS2 and PS2.SD): Blue, Red, Green and Near-Infrared band
      - Blue:  465 - 515 nm
      - Green: 547 - 585 nm
      - Red:   650 - 680 nm
      - Near Infrared: 845 - 885 nm
   - Eight-bandImager (PSB.SB) : the above four bands plus Coastal Blue, Green, Yellow, and Red Edge

- **Output format**: GeoTIFFs  

- **Image CRS**: EPSG:4326 (Geographic Latitude/Longitude)

- Link: https://developers.planet.com/docs/data/psscene/

2) RAPIDEYE	(5 meter resolution)

3) SKYSAT (50 centimeter resolution)

- Blue, Green, Red, Near Infrared, and Panchromatic bands
- Data availability	2014 -> now


Source: https://docs.sentinel-hub.com/api/latest/data/planet/planet-scope/


## Step to Search & Download Planet Imagery

- Set Up API Key in the Planet Explorer: https://www.planet.com/explorer​

- Define an area of interest (AOI) or bounding box.​

- Save the AOI coordinates to GeoJSON format to convert it to geometry object : http://geojson.io/.​

- Use the Geometry Filter to get images that overlap with  AOI.​

- Use the Date Range Filter to get images acquired within a date range.​

- Use the Cloud Cover Filter to filter images based on the cloud coverage.​

- Use the Combined Filter to combine Geo, Date, Cloud filters. ​

- Use the API request object to conduct a 'quick search' of the Planet catalog ( items and assets).​

- Extract and use the Item ID of the image you would like to download.​

- Activate the Data API to download the available images.​

- Once the asset status is active,  Image can be downloaded by making a GET request.

## Planet Downloader Script:
- Set up filters (Geo, Date, Cloud filters).
- Perform API requests.
- Extract  and use the Item ID.
- Activate the Data API and downloads images at the specified latitude & longitude.
- Retry failed downloads.
- Construct location to save the images.
- Log errors.

## Planet downloader script requires:

- Get and setup Planet API Key
- Image latitude: from previous pandas DataFrame
- Image longitude: from previous pandas DataFrame
- Minimum month and year: April-2016
- Maximum month and year: April-2017
- Zoom level = 16
- Maximum cloud filter: = 0.01 (1%)

### Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Add absolute path to the project folder

In [2]:
import sys

sys.path.append("/content/drive/MyDrive/UNECA_MachineLearning_Project/")

# See the full list of paths in sys.path
sys.path

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/UNECA_MachineLearning_Project/']

### Importing necessary python libraries and modules

In [3]:
# Provides mathematical functions like sqrt, sin, cos,
import math

# Used for making HTTP requests to APIs
import requests

# Allows using basic auth in requests for APIs that need username/password.
from requests.auth import HTTPBasicAuth

# Provides operating system related functions like path management
import os

# Used for working with JSON data.
import json

# Contains Python's input/output interfaces.
# BytesIO is used for in-memory file representations.
from io import BytesIO

# Useful for geospatial operations and geometry processing.
from shapely.geometry import Polygon

# Provides DataFrame and tabular data functionality.
import pandas as pd

# Import random library for generating random numbers
import random

# Import Numpy for numeric operations
import numpy as np

# Import GDAL/OSR for Geospatial data analysis
from osgeo import gdal, osr

# Import tqdm for progress bar visualization
from tqdm.notebook import tqdm

# Import Matplotlib for plotting
import matplotlib.pyplot as plt

# Import logging for logging capabilities
import logging

# Import time for time related operations
import time

### Add Base Directory

In [4]:
# Sets the base directory variable
BASE_DIR = '/content/drive/MyDrive/UNECA_MachineLearning_Project/'

In [5]:
# Change the current working directory
os.chdir(BASE_DIR)

# Print the current working directory to verify the change
print("Current Working Directory:", os.getcwd())

Current Working Directory: /content/drive/MyDrive/UNECA_MachineLearning_Project


### Define the folder data paths

In [6]:
# Creates the BASE_DIR variable & join it to the BASE_DIR
COUNTRIES_DIR = os.path.join(BASE_DIR, 'countries')

# Creates the PROCESSED_DIR variable & join it to the BASE_DIR
PROCESSED_DIR = os.path.join(BASE_DIR, 'processed')

###  Create directories in the current working directory

In [7]:
!mkdir -p $COUNTRIES_DIR/malawi_2016/images/

In [8]:
# Add the Planet api token
ACCESS_TOKEN_DIR = os.path.join(BASE_DIR, 'planet_api_key.txt')
ACCESS_TOKEN_DIR

'/content/drive/MyDrive/UNECA_MachineLearning_Project/planet_api_key.txt'

In [9]:
class PlanetDownloader:
    def __init__(self, api_key, item_type='PSScene'):
        self.api_key = api_key
        self.item_type = item_type

    def create_cords(lat, lon, zoom):
        xtile, ytile = deg_to_tile(lat, lon, zoom)

        coords = [tilexy_to_deg(xtile, ytile, zoom, a, b) for a,b in [(0,0), (0,255), (255,255), (255,0)]]
        return [[b,a] for a,b in coords]

    def download_image(self, lat, lon, min_year, min_month, max_year, max_month, zoom=16, cloud_max=0.05):
        '''
        Use this method to download an image at a lat, lon in some time range
        If multiple images are available, the latest is downloaded

        I would not increase zoom
        cloud_max is the maximum cloud filter, defaulting to 5%
        '''
        assert 0 <= cloud_max <= 1.0
        if min_month < 10:
            min_month = '0' + str(min_month)

        if max_month < 10:
            max_month = '0' + str(max_month)

        cords = PlanetDownloader.create_cords(lat, lon, zoom)
        geo_json_geometry = {
          "type": "Polygon",
          "coordinates": [
              cords
          ],

        }

        # filter for items the overlap with our chosen geometry
        geometry_filter = {
          "type": "GeometryFilter",
          "field_name": "geometry",
          "config": geo_json_geometry,
        }

        # filter images acquired in a certain date range
        date_range_filter = {
          "type": "DateRangeFilter",
          "field_name": "acquired",
          "config": {
            "gte": "{}-{}-01T00:00:00.000Z".format(min_year, min_month),
            "lte": "{}-{}-31T00:00:00.000Z".format(max_year, max_month)
          }
        }

        # filter any images which are more than 50% clouds
        cloud_cover_filter = {
          "type": "RangeFilter",
          "field_name": "cloud_cover",
          "config": {
            "lte": cloud_max
          }
        }

        # create a filter that combines our geo and date filters
        # could also use an "OrFilter"
        reservoir = {
          "type": "AndFilter",
          "config": [geometry_filter, date_range_filter, cloud_cover_filter]
        }

        # Search API request object
        search_endpoint_request = {
          "item_types": [self.item_type],
          "filter": reservoir
        }

        result = \
          requests.post(
            'https://api.planet.com/data/v1/quick-search',
            auth=HTTPBasicAuth(self.api_key, ''),
            json=search_endpoint_request)

        geojson = result.json()

        x, y = deg_to_tile(lat, lon, zoom)
        item_id = None

        #features = geojson["features"]
        #if len(features) == 0:
        #   return None

        #print("GeoJSON Response:", geojson)
        if len(geojson['features']) == 0:
          #print('No image found, try widening your search or using a different satellite')
          return None
        else:
            # planet for some reason will return results that don't even contain the requested geometry -_-
            # this will look for the LATEST (closest to the max time) match that actually contains our geometry
            polya = Polygon(cords)
            b_cords = [tilexy_to_deg(x,y,zoom,a,b) for a,b in [(0,0), (1,0), (1,1), (0,1)]]
            polyb = Polygon([(b,a) for (a,b) in b_cords])

            for idx in range(len(geojson["features"]) - 1, -1, -1):
                polyc = Polygon(geojson["features"][idx]['geometry']['coordinates'][0])

                if polyc.contains(polya) and polyc.contains(polyb):
                    item_id = geojson["features"][idx]['id']
                    break

        if item_id is None:
            # print('No good images found')
            return None

        url = 'https://tiles0.planet.com/data/v1/{}/{}/{}/{}/{}.png?api_key={}'.format(self.item_type, item_id, zoom, x, y, self.api_key)

        res = requests.get(url)
        if res.status_code >= 400:
            # print('download error')
            return None

        return plt.imread(BytesIO(res.content))


"""
Important geoconversion functions
"""

def tilexy_to_deg(xtile, ytile, zoom, x, y):
    """Converts a specific location on a tile (x,y) to geocoordinates."""
    decimal_x = xtile + x / 256
    decimal_y = ytile + y / 256
    n = 2.0 ** zoom
    lon_deg = decimal_x / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * decimal_y / n)))
    lat_deg = math.degrees(lat_rad)
    return (lat_deg, lon_deg)

def deg_to_tilexy(lat_deg, lon_deg, zoom):
    """Converts geocoordinates to an x,y position on a tile."""
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    x = ((lon_deg + 180.0) / 360.0 * n)
    y = ((1.0 - math.log(math.tan(lat_rad) + (1 / math.cos(lat_rad)))
        / math.pi) / 2.0 * n)
    return (int((x % 1) * 256), int((y % 1) * 256))

def tile_to_deg(xtile, ytile, zoom):
    """Returns the coordinates of the northwest corner of a Slippy Map
    x,y tile"""
    n = 2.0 ** zoom
    lon_deg = xtile / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
    lat_deg = math.degrees(lat_rad)
    return (lat_deg, lon_deg)

def deg_to_tile(lat_deg, lon_deg, zoom):
    """Converts coordinates into the nearest x,y Slippy Map tile"""
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.log(math.tan(lat_rad) + (1 / math.cos(lat_rad)))
                 / math.pi) / 2.0 * n)
    return (xtile, ytile)

In [10]:
df_download = pd.read_csv(os.path.join(PROCESSED_DIR, 'df_malawi_loc_labed.csv'))
df_download

,image_name,image_lat,image_lon,cluster_lat,cluster_lon,cons_pc,nightlights,nightlights_bin
0,-17.140065764205975_35.17229723579403_-17.0951...,-17.140066,35.172297,-17.095150,35.217213,1.423239,0.025206,1
1,-17.125093842803985_35.17229723579403_-17.0951...,-17.125094,35.172297,-17.095150,35.217213,1.423239,0.025206,1
2,-17.11012192140199_35.17229723579403_-17.09515...,-17.110122,35.172297,-17.095150,35.217213,1.423239,0.025206,1
3,-17.09515_35.17229723579403_-17.09515_35.21721...,-17.095150,35.172297,-17.095150,35.217213,1.423239,0.025206,1
4,-17.08017807859801_35.17229723579403_-17.09515...,-17.080178,35.172297,-17.095150,35.217213,1.423239,0.025206,1
...,...,...,...,...,...,...,...,...
33895,-9.429667_33.06703376420597_-9.429667_33.02211...,-9.429667,33.067034,-9.429667,33.022118,1.534702,0.000448,0
33896,-9.414695078598008_33.06703376420597_-9.429667...,-9.414695,33.067034,-9.429667,33.022118,1.534702,0.000448,0
33897,-9.399723157196016_33.06703376420597_-9.429667...,-9.399723,33.067034,-9.429667,33.022118,1.534702,0.000448,0
33898,-9.384751235794024_33.06703376420597_-9.429667...,-9.384751,33.067034,-9.429667,33.022118,1.534702,0.000448,0


## Assign the country code to the country column

In [11]:
df_download['country'] = 'mw'
df_download

,image_name,image_lat,image_lon,cluster_lat,cluster_lon,cons_pc,nightlights,nightlights_bin,country
0,-17.140065764205975_35.17229723579403_-17.0951...,-17.140066,35.172297,-17.095150,35.217213,1.423239,0.025206,1,mw
1,-17.125093842803985_35.17229723579403_-17.0951...,-17.125094,35.172297,-17.095150,35.217213,1.423239,0.025206,1,mw
2,-17.11012192140199_35.17229723579403_-17.09515...,-17.110122,35.172297,-17.095150,35.217213,1.423239,0.025206,1,mw
3,-17.09515_35.17229723579403_-17.09515_35.21721...,-17.095150,35.172297,-17.095150,35.217213,1.423239,0.025206,1,mw
4,-17.08017807859801_35.17229723579403_-17.09515...,-17.080178,35.172297,-17.095150,35.217213,1.423239,0.025206,1,mw
...,...,...,...,...,...,...,...,...,...
33895,-9.429667_33.06703376420597_-9.429667_33.02211...,-9.429667,33.067034,-9.429667,33.022118,1.534702,0.000448,0,mw
33896,-9.414695078598008_33.06703376420597_-9.429667...,-9.414695,33.067034,-9.429667,33.022118,1.534702,0.000448,0,mw
33897,-9.399723157196016_33.06703376420597_-9.429667...,-9.399723,33.067034,-9.429667,33.022118,1.534702,0.000448,0,mw
33898,-9.384751235794024_33.06703376420597_-9.429667...,-9.384751,33.067034,-9.429667,33.022118,1.534702,0.000448,0,mw


In [19]:
# Download images using Planet API

"""
Download images using a pandas DataFrame that has "image_lat", "image_lon", "image_name", "country" as columns

Saves images to the corresponding country's images folder

Use zoom = 16.
"""

def download_images(df):

    access = None
    with open(ACCESS_TOKEN_DIR, 'r') as f:
        access = f.readlines()[0]
    imd = PlanetDownloader(access)
    num_retries = 20
    wait_time = 0.1 # seconds

    # drops what is already downloaded
    already_downloaded = os.listdir(os.path.join(COUNTRIES_DIR, 'malawi_2016', 'images'))

    already_downloaded =  list(set(already_downloaded).intersection(set(df['image_name'])))
    print('Already downloaded ' + str(len(already_downloaded)))

    df = df.set_index('image_name').drop(already_downloaded).reset_index()
    print('Need to download ' + str(len(df)))

    # use three years of images to find one that matches search critera
    min_year = 2014
    min_month = 1
    max_year = 2016
    max_month = 12

    for _, r in tqdm(df.iterrows(), total=df.shape[0]):
        lat = r.image_lat
        lon = r.image_lon
        name = r.image_name
        country_dir = None
        if r.country == 'mw':
            country_dir = 'malawi_2016'
        else:
            print(f"unrecognized country: {r.country}")
            raise ValueError()
        image_save_path = os.path.join(COUNTRIES_DIR, country_dir, 'images', r.image_name)
        try:
            im = imd.download_image(lat, lon, min_year, min_month, max_year, max_month)
            if (type(im) == str and im == 'RETRY') or im is None:
                resolved = False
                for _ in range(num_retries):
                    time.sleep(wait_time)
                    im = imd.download_image(lat, lon, min_year, min_month, max_year, max_month)
                    if (type(im) == str and im == 'RETRY') or im is None:
                        continue
                    else:
                        plt.imsave(image_save_path, im)
                        resolved = True
                        break
                if not resolved:
                    print(f'Could not download {lat}, {lon} despite several retries and waiting')
                    continue
                else:
                    pass
            else:
                # no issues, save according to naming convention
                plt.imsave(image_save_path, im)

        except Exception as e:
            logging.error(f"Error-could not download {lat}, {lon}", exc_info=True)
            continue

In [21]:
download_images(df_download)

Already downloaded 33900
Need to download 0


0it [00:00, ?it/s]